In [1]:
import json
import csv
import pandas as pd
import numpy as np
import re
from pathlib import Path

In [2]:
cwd = Path.cwd()
rel_path = '../data/'
mod_path = Path('Full_Dataset_Normalizer.ipynb').parent
src_path_1 = (mod_path / rel_path).resolve()

In [3]:
#Universal Variables
imagetags = set()
hashtags = set()
imageobjects = set()
imagecolors = set()
fieldnames = ['PageName', 'PostID', 'PostTime', 'Height', 'Width', 'Followers', 'Comments', 'Likes']

In [4]:
def Column_Headers(imagetags, hashtags, imageobjects, imagecolors):
    with open('visit_utah_updated_01_19.json', mode = 'r', encoding = 'utf-8') as f:
        data = json.loads(f.read(), encoding = 'utf-8')
    #Get all the column header names from the data
    for photo in data: #Get the column names
        md = photo['metadata'] #The photo's metadata
        tags = md.get('imageTags', '') #The photo's image tags by the API
        ht = md.get('hashtags', '') #The photo's hash tags by the user
        objects = md.get('imageObjects','') #Objects in the photo found by the API
        colors = md.get('imageColors','') #The 3 main colors in the photo found by the API
        for tag in tags:
            imagetags.add(tag['value'])
        for tag in ht:
            hashtags.add(tag)
        for obj in objects:
            imageobjects.add(obj['name'])
        for color in colors:
            imagecolors.add(color['value'])
    return data

In [5]:
def Image_Tag_Data(fieldnames, data, imagetags):
    fn = fieldnames.copy()
    length = 8+len(imagetags)
    for tag in imagetags:
        fn.append(tag+'_tag')
    DF = pd.DataFrame()
    for photo in data: #Get the data out with the associated column name
        photodata = list(['0']*length)
        md = photo['metadata'] #The photo's metadata
        tags = md['imageTags'] #The photo's image tags by the API
        photodata[0] = photo['pageName']
        photodata[1] = photo['extPostId']
        photodata[2] = photo['extCreatedAt']
        photodata[3] = md['imgHeight']
        photodata[4] = md['imgWidth']
        photodata[5] = photo['nFollowers']
        photodata[6] = photo['nComments']
        photodata[7] = photo['nLikes']
        i = 8
        for tag1 in imagetags:
            for tag2 in tags:
                if tag1 == tag2['value']:
                    photodata[i] = 1 #tag2['confidence']
                    break
            i = i+1
        df_temp = pd.DataFrame(data = np.array(photodata).reshape((1,len(fn))), columns = fn)
        DF = DF.append(df_temp, ignore_index = True)
    DF1 = DF.iloc[:,0:8]
    DF2 = DF.iloc[:,8:]
    DF2 = DF2.astype(float)
    DF2 = DF2.round(decimals=0)
    DF2 = DF2.astype(int)
    sum1 = DF2.sum(axis=0)
    badCols = list()
    for index in range(len(sum1)):
        if sum1[index]  < 10:
            badCols.append(index)
    DF2 = DF2.drop(DF2.columns[badCols], axis=1)
    DF = pd.concat([DF1, DF2],axis=1)
    return DF

In [6]:
def Hashtag_Data(fieldnames, data, hashtags):
    fn = fieldnames.copy()
    length = 8+len(hashtags)
    for tag in hashtags:
        fn.append(tag+'_ht')
    DF = pd.DataFrame()
    for photo in data: #Get the data out with the associated column name
        photodata = list(['0']*length)
        md = photo['metadata'] #The photo's metadata
        ht = md['hashtags']
        photodata[0] = photo['pageName']
        photodata[1] = photo['extPostId']
        photodata[2] = photo['extCreatedAt']
        photodata[3] = md['imgHeight']
        photodata[4] = md['imgWidth']
        photodata[5] = photo['nFollowers']
        photodata[6] = photo['nComments']
        photodata[7] = photo['nLikes']
        i = 8
        for tag1 in hashtags:
            for tag2 in ht:
                if tag1 == tag2:
                    photodata[i] = 1
                    break
            i = i+1
        df_temp = pd.DataFrame(data = np.array(photodata).reshape((1,len(fn))), columns = fn)
        DF = DF.append(df_temp, ignore_index = True)
    DF1 = DF.iloc[:,0:8]
    DF2 = DF.iloc[:,8:]
    DF2 = DF2.astype(int)
    sum1 = DF2.sum(axis=0)
    badCols = list()
    for index in range(len(sum1)):
        if sum1[index]  < 10:
            badCols.append(index)
    DF2 = DF2.drop(DF2.columns[badCols], axis=1)
    DF = pd.concat([DF1, DF2],axis=1)
    return DF

In [7]:
def Image_Object_Data(fieldnames, data, imageobjects):    
    fn = fieldnames.copy()
    length = 8+len(imageobjects)
    for objects in imageobjects:
        fn.append(objects+'_obj')
    DF = pd.DataFrame()
    for photo in data: #Get the data out with the associated column name
        photodata = list(['0']*length)
        md = photo['metadata'] #The photo's metadata
        objects = md['imageObjects']
        photodata[0] = photo['pageName']
        photodata[1] = photo['extPostId']
        photodata[2] = photo['extCreatedAt']
        photodata[3] = md['imgHeight']
        photodata[4] = md['imgWidth']
        photodata[5] = photo['nFollowers']
        photodata[6] = photo['nComments']
        photodata[7] = photo['nLikes']
        i = 8
        for object1 in imageobjects:
            for object2 in objects:
                if object1 == object2['name']:
                    photodata[i] = 1 #object2['conf']
                    break
            i = i+1
        df_temp = pd.DataFrame(data = np.array(photodata).reshape((1,len(fn))), columns = fn)
        DF = DF.append(df_temp, ignore_index = True)
    str_obj = DF.iloc[:,0:8] #creates dataset of just str data
    int_obj = DF.iloc[:,8:]
    int_obj = int_obj.astype(int) #creates dataset to convert datatypes to int
    sum1 = int_obj.sum(axis=0) #get sums of columns
    badCols = list()
    for index in range(len(sum1)):
       if sum1[index]  < 5: #if object occurs less than 5 times, column name will be added to badCols list
           badCols.append(index)
    int_obj = int_obj.drop(int_obj.columns[badCols], axis=1) #names in badCols list will be dropped from original dataset
    DF= pd.concat([str_obj, int_obj], axis=1) #combines str & int datasets
    return DF

In [8]:
def Image_Color_Data(fieldnames, data, imagecolors):
    fn = fieldnames.copy()
    length = 8+len(imagecolors)
    for color in imagecolors:
        fn.append(color)
    DF = pd.DataFrame()
    for photo in data: #Get the data out with the associated column name
        photodata = list(['0']*length)
        md = photo['metadata'] #The photo's metadata
        colors = md['imageColors']
        photodata[0] = photo['pageName']
        photodata[1] = photo['extPostId']
        photodata[2] = photo['extCreatedAt']
        photodata[3] = md['imgHeight']
        photodata[4] = md['imgWidth']
        photodata[5] = photo['nFollowers']
        photodata[6] = photo['nComments']
        photodata[7] = photo['nLikes']
        i = 8
        for color1 in imagecolors:
            for color2 in colors:
                if color1 == color2['value']:
                    photodata[i] = color2['confidence']
                    break
            i = i+1
        df_temp = pd.DataFrame(data = np.array(photodata).reshape((1,len(fn))), columns = fn)
        DF = DF.append(df_temp, ignore_index = True)
    DF.columns = map(str.upper, DF.columns) #Capitalize all column names
#This Color Group csv is a separate file that needs to be read in
    with open('Color Group.csv', mode='r') as infile:
        reader = csv.reader(infile)
        mydict = {rows[0]:rows[1] for rows in reader}
    DF=DF.rename(index=str,columns=mydict)
#create a list of color groups
    color=['Pink','Purple','Red','Orange','Yellow','Green','Cyan','Blue','Brown','White','Grey','Black']
#change value tpe to float
    for c in color:
        DF[c]=DF[c].astype('float')
    DF.head()
    df=DF.transpose().reset_index().rename(columns={'index':'Color'}) #transpose dataset 
    df = df.groupby('Color').sum() #groupby based on color group
    DF=df.T #transpose back
    DF.head()
    cols = ['PageName', 'PostID', 'PostTime','Height','Width','Followers', 'Comments', 'Likes', 'Pink','Purple','Red','Orange','Yellow','Green','Cyan','Blue','Brown','White','Grey','Black']
    DF = DF.reindex(columns=cols)
    return DF

In [9]:
def sylco(word):  #Syllable counter given only the spelling of a word
    word = word.lower()

    # exception_add are words that need extra syllables
    # exception_del are words that need less syllables

    exception_add = ['serious','crucial']
    exception_del = ['fortunately','unfortunately', 'facebook']

    co_one = ['cool','coach','coat','coal','count','coin','coarse','coup','coif','cook','coign','coiffe','coof','court']
    co_two = ['coapt','coed','coinci']

    pre_one = ['preach']

    syls = 0 #added syllable number
    disc = 0 #discarded syllable number

    #1) if letters < 3 : return 1
    if len(word) <= 3 :
        syls = 1
        return syls

    #2) if doesn't end with "ted" or "tes" or "ses" or "ied" or "ies", discard "es" and "ed" at the end.
    # if it has only 1 vowel or 1 set of consecutive vowels, discard. (like "speed", "fled" etc.)

    if word[-2:] == "es" or word[-2:] == "ed" :
        doubleAndtripple_1 = len(re.findall(r'[eaoui][eaoui]',word))
        if doubleAndtripple_1 > 1 or len(re.findall(r'[eaoui][^eaoui]',word)) > 1 :
            if word[-3:] == "ted" or word[-3:] == "tes" or word[-3:] == "ses" or word[-3:] == "ied" or word[-3:] == "ies" :
                pass
            else :
                disc+=1

    #3) discard trailing "e", except where ending is "le"  

    le_except = ['whole','mobile','pole','male','female','hale','pale','tale','sale','aisle','whale','while']

    if word[-1:] == "e" :
        if word[-2:] == "le" and word not in le_except :
            pass

        else :
            disc+=1

    #4) check if consecutive vowels exists, triplets or pairs, count them as one.

    doubleAndtripple = len(re.findall(r'[eaoui][eaoui]',word))
    tripple = len(re.findall(r'[eaoui][eaoui][eaoui]',word))
    disc+=doubleAndtripple + tripple

    #5) count remaining vowels in word.
    numVowels = len(re.findall(r'[eaoui]',word))

    #6) add one if starts with "mc"
    if word[:2] == "mc" :
        syls+=1

    #7) add one if ends with "y" but is not surrouned by vowel
    if word[-1:] == "y" and word[-2] not in "aeoui" :
        syls +=1

    #8) add one if "y" is surrounded by non-vowels and is not in the last word.

    for i,j in enumerate(word) :
        if j == "y" :
            if (i != 0) and (i != len(word)-1) :
                if word[i-1] not in "aeoui" and word[i+1] not in "aeoui" :
                    syls+=1

    #9) if starts with "tri-" or "bi-" and is followed by a vowel, add one.

    if word[:3] == "tri" and word[3] in "aeoui" :
        syls+=1

    if word[:2] == "bi" and word[2] in "aeoui" :
        syls+=1

    #10) if ends with "-ian", should be counted as two syllables, except for "-tian" and "-cian"

    if word[-3:] == "ian" : 
    #and (word[-4:] != "cian" or word[-4:] != "tian") :
        if word[-4:] == "cian" or word[-4:] == "tian" :
            pass
        else :
            syls+=1

    #11) if starts with "co-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:2] == "co" and word[2] in 'eaoui' :

        if word[:4] in co_two or word[:5] in co_two or word[:6] in co_two :
            syls+=1
        elif word[:4] in co_one or word[:5] in co_one or word[:6] in co_one :
            pass
        else :
            syls+=1

    #12) if starts with "pre-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:3] == "pre" and word[3] in 'eaoui' :
        if word[:6] in pre_one :
            pass
        else :
            syls+=1

    #13) check for "-n't" and cross match with dictionary to add syllable.

    negative = ["doesn't", "isn't", "shouldn't", "couldn't","wouldn't"]

    if word[-3:] == "n't" :
        if word in negative :
            syls+=1
        else :
            pass   

    #14) Handling the exceptional words.

    if word in exception_del :
        disc+=1

    if word in exception_add :
        syls+=1     

    # calculate the output
    return numVowels - disc + syls

In [10]:
def Readability_Data(fieldnames, data):
    fn = fieldnames.copy()
    length = 9
    fn.append('Flesch RE')
    DF = pd.DataFrame()
    for photo in data: #Get the data out with the associated column name
        text = photo['rawText'].replace('\u2063','')
        text = text.lower()
        text = re.sub(r'[^a-z\s#@]','',text) #Remove everything that isn't a letter or a # or @
        words = text.split()
        i = 0
        while i < len(words):
            if words[i][0] == '#' or words[i][0] == '@': #Remove # and @ words
                words.pop(i)
                i = i - 1
            i = i + 1
        photodata = list(['0']*length)
        md = photo['metadata'] #The photo's metadata
        words = words
        syllb = 0.0
        row = np.array([[0,0,0,0,0]])
        for word in words:
            syllb += sylco(word)
        photodata[0] = photo['pageName']
        photodata[1] = photo['extPostId']
        photodata[2] = photo['extCreatedAt']
        photodata[3] = md['imgHeight']
        photodata[4] = md['imgWidth']
        photodata[5] = photo['nFollowers']
        photodata[6] = photo['nComments']
        photodata[7] = photo['nLikes']
        if len(words) == 0:
            photodata[8] = 'NULL'
        else:
            photodata[8] = 206.835-1.015*len(words)-84.6*(syllb/len(words)) #Not a number when it goes back into the dataframe
        df_temp = pd.DataFrame(data = np.array(photodata).reshape((1,len(fn))), columns = fn)
        DF = DF.append(df_temp, ignore_index = True)
    return DF

In [11]:
def PHash_Data(fieldnames, data):
    fn = fieldnames.copy()
    length = 9
    fn.append('PHash')
    DF = pd.DataFrame()
    for photo in data: #Get the data out with the associated column name
        photodata = list(['0']*length)
        md = photo['metadata'] #The photo's metadata
        photodata[0] = photo['pageName']
        photodata[1] = photo['extPostId']
        photodata[2] = photo['extCreatedAt']
        photodata[3] = md['imgHeight']
        photodata[4] = md['imgWidth']
        photodata[5] = photo['nFollowers']
        photodata[6] = photo['nComments']
        photodata[7] = photo['nLikes']
        photodata[8] = md['imagePHash']
        df_temp = pd.DataFrame(data = np.array(photodata).reshape((1,len(fn))), columns = fn)
        DF = DF.append(df_temp, ignore_index = True)
    return DF

In [12]:
def DateTime_Parts(data): #Creates a bunch of different date times in boolean (0/1) form. Only read in Final data with a "PostTime" variable
    data['date_series'] = pd.to_datetime(data['PostTime'])
    data['yearofpost'] = data['date_series'].dt.year
    data['monthofpost'] = data['date_series'].dt.month
    data['hourofpost'] = data['date_series'].dt.hour
    data['minuteofpost'] = data['date_series'].dt.minute
    data['quarterofpost'] = data['date_series'].dt.quarter
    data['JanuaryPost'] = np.where(data['monthofpost'] == 1, 1,0)
    data['FebuaryPost'] = np.where(data['monthofpost'] == 2, 1,0)
    data['MarchPost'] = np.where(data['monthofpost'] == 3, 1,0)
    data['AprilPost'] = np.where(data['monthofpost'] == 4, 1,0)
    data['MayPost'] = np.where(data['monthofpost'] == 5, 1,0)
    data['JunePost'] = np.where(data['monthofpost'] == 6, 1,0)
    data['JulyPost'] = np.where(data['monthofpost'] == 7, 1,0)
    data['AugustPost'] = np.where(data['monthofpost'] == 8, 1,0)
    data['SeptemberPost'] = np.where(data['monthofpost'] == 9, 1,0)
    data['OctoberPost'] = np.where(data['monthofpost'] == 10, 1,0)
    data['NovemberPost'] = np.where(data['monthofpost'] == 11, 1,0)
    data['DecemberPost'] = np.where(data['monthofpost'] == 12, 1,0)
    conditions = [
        (data['hourofpost'] <= 5) | (data['hourofpost'] > 22),
        (data['hourofpost'] > 5) & (data['hourofpost'] <= 12),
        (data['hourofpost'] > 12) & (data['hourofpost'] <= 17),
        (data['hourofpost'] > 17) & (data['hourofpost'] <= 22)]
    choices = ['11pm to 5am', '6am to 12pm', '1pm to 5pm', '6pm to 10pm']
    data['TimeofDay'] = np.select(conditions, choices, default='N/A')
    data['11pm to 5am'] = np.where(data['TimeofDay']=='11pm to 5am',1,0)
    data['6am to 12pm'] = np.where(data['TimeofDay']=='6am to 12pm',1,0)
    data['1pm to 5pm'] = np.where(data['TimeofDay']=='1pm to 5pmm',1,0)
    data['6pm to 10pm'] = np.where(data['TimeofDay']=='6pm to 10pm',1,0)
    data = data.drop(columns = ['TimeofDay'])
    DF = data
    return DF

In [13]:
def SchoolPart(df):
    df['Flesch RE'] = pd.to_numeric(df['Flesch RE'])
    conditions = [
        df['Flesch RE'] >= 100,
        (df['Flesch RE'] < 100) & (df['Flesch RE'] >= 90),
        (df['Flesch RE'] < 90) & (df['Flesch RE'] >= 80),
        (df['Flesch RE'] < 80) & (df['Flesch RE'] >= 70),
        (df['Flesch RE'] < 70) & (df['Flesch RE'] >= 60),
        (df['Flesch RE'] < 60) & (df['Flesch RE'] >= 50),
        (df['Flesch RE'] < 50) & (df['Flesch RE'] >= 30),
        (df['Flesch RE'] < 30) & (df['Flesch RE'] >= 0),
        df['Flesch RE'] < 0]
    choices = ['4th or Below', '5th', '6th', '7th', '8th or 9th', '10th - 12th', 'College', 'College Graduate', 'Post Graduate Studies']
    df['SchoolYearRead'] = np.select(conditions, choices, default='N/A')
    conditions = [
        df['Flesch RE'] >= 90,
        (df['Flesch RE'] < 90) & (df['Flesch RE'] >= 60),
        (df['Flesch RE'] < 60) & (df['Flesch RE'] >= 50),
        (df['Flesch RE'] < 50) & (df['Flesch RE'] >= 30),
        (df['Flesch RE'] < 30) & (df['Flesch RE'] >= 0),
        df['Flesch RE'] < 0]
    choices = ['Elementary and Below', 'Middle', 'High', 'College', 'College Graduate', 'Post Graduate Studies']
    df['SchoolGroupRead'] = np.select(conditions, choices, default='N/A')
    return df

In [14]:
#Used to see all column names in output display
pd.set_option('display.max_columns', 500)

In [15]:
photos = Column_Headers(imagetags, hashtags, imageobjects, imagecolors)
get_tag = Image_Tag_Data(fieldnames, photos, imagetags)
get_hash = Hashtag_Data(fieldnames, photos, hashtags)
get_obj = Image_Object_Data(fieldnames, photos, imageobjects)
get_color = Image_Color_Data(fieldnames, photos, imagecolors)
get_readability = Readability_Data(fieldnames, photos)
get_phash = PHash_Data(fieldnames, photos)

In [16]:
DFmerge1 = pd.merge(get_tag, get_hash, how = 'left', on = ['PageName', 'PostID', 'PostTime', 'Height', 'Width', 'Followers', 'Comments', 'Likes'])
DFmerge2 = pd.merge(DFmerge1, get_obj, how = 'left', on = ['PageName', 'PostID', 'PostTime', 'Height', 'Width', 'Followers', 'Comments', 'Likes'])
DFmerge3 = pd.merge(DFmerge2, get_color, how = 'left', on = ['PageName', 'PostID', 'PostTime', 'Height', 'Width', 'Followers', 'Comments', 'Likes'])
DFmerge4 = pd.merge(DFmerge3, get_readability, how = 'left', on = ['PageName', 'PostID', 'PostTime', 'Height', 'Width', 'Followers', 'Comments', 'Likes'])
DFmerge5 = pd.merge(DFmerge4, get_hash, how = 'left', on = ['PageName', 'PostID', 'PostTime', 'Height', 'Width', 'Followers', 'Comments', 'Likes'])

In [17]:
DFmerge5['Followers'] = pd.to_numeric(DFmerge5['Followers'])
DFmerge5['Likes'] = pd.to_numeric(DFmerge5['Likes'])
DFmerge5['Comments'] = pd.to_numeric(DFmerge5['Comments'])
DFmerge5['engagement_rate'] = ((DFmerge5['Comments'] + DFmerge5['Likes']) / DFmerge5['Followers']) * 100
DFmerge5 = SchoolPart(DFmerge5)
DFmerge5.head()

,PageName,PostID,PostTime,Height,Width,Followers,Comments,Likes,shrubland_tag,stream_tag,vacation_tag,landmark_tag,winter sport_tag,formation_tag,sea_tag,lake_tag,tree_tag,horizon_tag,watercourse_tag,autumn_tag,field_tag,skiing_tag,extreme sport_tag,wilderness_tag,mountain range_tag,wadi_tag,ski equipment_tag,vegetation_tag,fun_tag,national park_tag,mount scenery_tag,adventure_tag,outcrop_tag,waterfall_tag,recreation_tag,ski_tag,arch_tag,butte_tag,glacial landform_tag,leaf_tag,tourist attraction_tag,narrows_tag,piste_tag,atmosphere_tag,coast_tag,river_tag,geological phenomenon_tag,historic site_tag,freezing_tag,sunrise_tag,plant_tag,outer space_tag,morning_tag,water feature_tag,arctic_tag,alps_tag,hill station_tag,vehicle_tag,snow_tag,dawn_tag,sky_tag,ecoregion_tag,path_tag,winter_tag,mountain_tag,loch_tag,highland_tag,cliff_tag,badlands_tag,ridge_tag,ecosystem_tag,archaeological site_tag,grass_tag,mountainous landforms_tag,darkness_tag,night_tag,grassland_tag,calm_tag,soil_tag,star_tag,water resources_tag,ice_tag,rock_tag,shore_tag,canyon_tag,nature_tag,astronomical object_tag,escarpment_tag,landscape_tag,terrain_tag,tourism_tag,massif_tag,body of water_tag,water_tag,cloud_tag,nature reserve_tag,reflection_tag,geology_tag,phenomenon_tag,sunset_ht_x,camping_ht_x,travel_ht_x,mountainbiking_ht_x,findyourgreatest_ht_x,roadtrip_ht_x,mighty5_ht_x,waterfallwednesday_ht_x,getoutside_ht_x,beutahful_ht_x,adventure_ht_x,lifeelevated_ht_x,waterfall_ht_x,exploremore_ht_x,explore_ht_x,fishing_ht_x,hike_ht_x,winter_ht_x,skiutah_ht_x,findyourpark_ht_x,wanderlust_ht_x,summer_ht_x,utah_ht_x,visitutah_ht_x,scenery_ht_x,hiking_ht_x,Castle_obj,Tent_obj,Human face_obj,Fish_obj,Helmet_obj,House_obj,Land vehicle_obj,Person_obj,Woman_obj,Car_obj,Bird_obj,Fountain_obj,Animal_obj,Cattle_obj,Street light_obj,Bicycle wheel_obj,Poster_obj,Footwear_obj,Dog_obj,Bicycle_obj,Building_obj,Man_obj,Sports equipment_obj,Tree_obj,Wheel_obj,Vehicle_obj,Flower_obj,Tire_obj,Snowboard_obj,Clothing_obj,Plant_obj,Ski_obj,Parachute_obj,Sculpture_obj,Motorcycle_obj,Bicycle helmet_obj,Boat_obj,Pink,Purple,Red,Orange,Yellow,Green,Cyan,Blue,Brown,White,Grey,Black,Flesch RE,sunset_ht_y,camping_ht_y,travel_ht_y,mountainbiking_ht_y,findyourgreatest_ht_y,roadtrip_ht_y,mighty5_ht_y,waterfallwednesday_ht_y,getoutside_ht_y,beutahful_ht_y,adventure_ht_y,lifeelevated_ht_y,waterfall_ht_y,exploremore_ht_y,explore_ht_y,fishing_ht_y,hike_ht_y,winter_ht_y,skiutah_ht_y,findyourpark_ht_y,wanderlust_ht_y,summer_ht_y,utah_ht_y,visitutah_ht_y,scenery_ht_y,hiking_ht_y,engagement_rate,SchoolYearRead,SchoolGroupRead
0,visitutah,1967170976682576484,2019-01-29T01:24:36.000Z,800,1064,86200,18,1927,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.157394,0,0,0,0.0846758,0,0.125012,0,0,0,0,68.095000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2.256381,8th or 9th,Middle
1,visitutah,1965520773869116669,2019-01-26T18:45:56.000Z,1350,1080,85978,160,14462,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0777762,0,0.0680237,0,0,0.123582,0,3.345000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,17.006676,College Graduate,College Graduate
2,visitutah,1964829982460857211,2019-01-25T19:53:28.000Z,812,1080,85929,42,4336,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0

In [30]:
DF = DFmerge5[DFmerge5['Followers'] > 0]
DF_top = DF[DF['engagement_rate'] > 10]
DF_dummies = pd.get_dummies(DF_top['SchoolGroupRead'])
DF_top = DF_top.drop(['PageName', 'PostID', 'PostTime', 'Height', 'Width', 'SchoolYearRead', 'SchoolGroupRead'], axis=1)
df = pd.concat([DF_dummies, DF_top], axis=1)

In [35]:
df.head()

,College,College Graduate,Elementary and Below,High,Middle,Followers,Comments,Likes,shrubland_tag,stream_tag,vacation_tag,landmark_tag,winter sport_tag,formation_tag,sea_tag,lake_tag,tree_tag,horizon_tag,watercourse_tag,autumn_tag,field_tag,skiing_tag,extreme sport_tag,wilderness_tag,mountain range_tag,wadi_tag,ski equipment_tag,vegetation_tag,fun_tag,national park_tag,mount scenery_tag,adventure_tag,outcrop_tag,waterfall_tag,recreation_tag,ski_tag,arch_tag,butte_tag,glacial landform_tag,leaf_tag,tourist attraction_tag,narrows_tag,piste_tag,atmosphere_tag,coast_tag,river_tag,geological phenomenon_tag,historic site_tag,freezing_tag,sunrise_tag,plant_tag,outer space_tag,morning_tag,water feature_tag,arctic_tag,alps_tag,hill station_tag,vehicle_tag,snow_tag,dawn_tag,sky_tag,ecoregion_tag,path_tag,winter_tag,mountain_tag,loch_tag,highland_tag,cliff_tag,badlands_tag,ridge_tag,ecosystem_tag,archaeological site_tag,grass_tag,mountainous landforms_tag,darkness_tag,night_tag,grassland_tag,calm_tag,soil_tag,star_tag,water resources_tag,ice_tag,rock_tag,shore_tag,canyon_tag,nature_tag,astronomical object_tag,escarpment_tag,landscape_tag,terrain_tag,tourism_tag,massif_tag,body of water_tag,water_tag,cloud_tag,nature reserve_tag,reflection_tag,geology_tag,phenomenon_tag,sunset_ht_x,camping_ht_x,travel_ht_x,mountainbiking_ht_x,findyourgreatest_ht_x,roadtrip_ht_x,mighty5_ht_x,waterfallwednesday_ht_x,getoutside_ht_x,beutahful_ht_x,adventure_ht_x,lifeelevated_ht_x,waterfall_ht_x,exploremore_ht_x,explore_ht_x,fishing_ht_x,hike_ht_x,winter_ht_x,skiutah_ht_x,findyourpark_ht_x,wanderlust_ht_x,summer_ht_x,utah_ht_x,visitutah_ht_x,scenery_ht_x,hiking_ht_x,Castle_obj,Tent_obj,Human face_obj,Fish_obj,Helmet_obj,House_obj,Land vehicle_obj,Person_obj,Woman_obj,Car_obj,Bird_obj,Fountain_obj,Animal_obj,Cattle_obj,Street light_obj,Bicycle wheel_obj,Poster_obj,Footwear_obj,Dog_obj,Bicycle_obj,Building_obj,Man_obj,Sports equipment_obj,Tree_obj,Wheel_obj,Vehicle_obj,Flower_obj,Tire_obj,Snowboard_obj,Clothing_obj,Plant_obj,Ski_obj,Parachute_obj,Sculpture_obj,Motorcycle_obj,Bicycle helmet_obj,Boat_obj,Pink,Purple,Red,Orange,Yellow,Green,Cyan,Blue,Brown,White,Grey,Black,Flesch RE,sunset_ht_y,camping_ht_y,travel_ht_y,mountainbiking_ht_y,findyourgreatest_ht_y,roadtrip_ht_y,mighty5_ht_y,waterfallwednesday_ht_y,getoutside_ht_y,beutahful_ht_y,adventure_ht_y,lifeelevated_ht_y,waterfall_ht_y,exploremore_ht_y,explore_ht_y,fishing_ht_y,hike_ht_y,winter_ht_y,skiutah_ht_y,findyourpark_ht_y,wanderlust_ht_y,summer_ht_y,utah_ht_y,visitutah_ht_y,scenery_ht_y,hiking_ht_y,engagement_rate
1,0,1,0,0,0,85978,160,14462,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0777762,0,0.0680237,0,0,0.123582,0,3.345,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,17.006676
3,0,0,1,0,0,85857,16,9191,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.361114,0.126402,0,0,0.113819,97.025,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,10.723645
4,0,0,0,0,1,85787,49,21375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.371835,0,74.805,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,24.973481
5,1,0,0,0,0,85553,53,24031,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0

In [32]:
DF_trans = np.transpose(df)
DF_trans['sum'] = np.sum(DF_trans,axis=1)

In [33]:
DF_trans

,1,3,4,5,6,10,13,14,20,21,24,26,28,31,46,47,50,54,58,60,94,sum
College,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,3.000000e+00
College Graduate,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.000000e+00
Elementary and Below,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2.000000e+00
High,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.000000e+00
Middle,0,0,1,0,0,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1,1,1.300000e+01
Followers,85978,85857,85787,85553,85332,85073,84987,84983,84564,84564,84351,84288,84171,84052,83212,83140,82998,82807,82475,82256,79627,1.766055e+06
Comments,160,16,49,53,60,70,32,25,28,57,51,33,32,128,117,73,9,34,65,62,283,1.437000e+03
Likes,14462,9191,21375,24031,22072,33023,24062,10522,16875,49192,9989,20150,12677,18716,50124,38953,8851,13916,24766,18322,85484,5.267530e+05
shrubland_tag,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000e+00
stream_tag,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000e+00


In [36]:
DF_trans.sort_values(by=['sum'], ascending=False)

,1,3,4,5,6,10,13,14,20,21,24,26,28,31,46,47,50,54,58,60,94,sum
Followers,85978,85857,85787,85553,85332,85073,84987,84983,84564,84564,84351,84288,84171,84052,83212,83140,82998,82807,82475,82256,79627,1.766055e+06
Likes,14462,9191,21375,24031,22072,33023,24062,10522,16875,49192,9989,20150,12677,18716,50124,38953,8851,13916,24766,18322,85484,5.267530e+05
Comments,160,16,49,53,60,70,32,25,28,57,51,33,32,128,117,73,9,34,65,62,283,1.437000e+03
Flesch RE,3.345,97.025,74.805,32.505,15.64,61.325,82.39,71.815,57.2336,95.0271,63.695,84.9,71.7816,88.905,85.0741,61.665,39.33,41.8518,70.1447,68.095,89.5169,1.356070e+03
engagement_rate,17.0067,10.7236,24.9735,28.151,25.9363,38.8995,28.3502,12.4107,19.9884,58.2387,11.9026,23.9453,15.099,22.4195,60.3771,46.9401,10.675,16.8464,30.1073,22.3497,107.711,6.330517e+02
visitutah_ht_y,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2.100000e+01
visitutah_ht_x,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2.100000e+01
Tree_obj,1,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1.700000e+01
Middle,0,0,1,0,0,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1,1,1.300000e+01
geological phenomenon_tag,0,0,0,0,1,0,1,0,1,1,0,0,1,1,1,1,0,0,0,0,1,9.000000e+00
